In [76]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webelement import WebElement
from webdriver_manager.chrome import ChromeDriverManager
from typing import List, Tuple

In [77]:
# Instantiate a Chrome driver.
def load_driver() -> None:
    # Load and configure webdriver.
    options: Options = Options()
    # Stop browser windows from actually popping up.
    options.add_argument('--headless')
    # Install a browser for use by Selenium.
    service: Service = Service(executable_path=ChromeDriverManager().install())
    return Chrome(service=service, options=options)

In [78]:
# Helper function that navigates to the search result page for a particular
# month on the USCF page for historical tournament data.
def navigate_to_uscf_page(driver: Chrome, date_to_visit: str) -> None:
    # Navigate to US Chess tournament search page.
    USCF_URL = 'http://www.uschess.org/datapage/events-rated.php'
    driver.get(USCF_URL)
    date_search_box: WebElement = driver.find_element('name', 'month')
    date_search_box.clear()
    date_search_box.send_keys(date_to_visit)

    # Select CA as the State Code, which is where all 
    # chess.com USCF tournaments are registered.
    state_search_box: WebElement = driver.find_element('name', 'states')
    state_search_box.clear()
    state_search_box.send_keys('CA')
    state_search_box.submit()

In [79]:
# Scrapes tournment urls for the month that the driver is currently
# pointed add. Helper function navigate_to_uscf_page() navigates to the
# correct page.
def scrape_uscf_tournament_urls(driver: Chrome) -> List[str]:
    table_body: List[WebElement] = driver.find_elements(By.TAG_NAME, 'tbody')[2]
    table_body_row: List[WebElement] = table_body.find_elements(By.TAG_NAME, 'tr')
    url_list: List[str] = []
    for row in table_body_row:
        # Each row is a WebElement with data about one tournament.
        table_row: List[WebElement] = row.find_elements(By.TAG_NAME, 'td')
        if len(table_row) >= 3:
            url: str = None
            for element in table_row:
                if element.text.isnumeric() and len(element.text) > 10:
                    url = element.find_element(By.TAG_NAME, 'a').get_attribute('href')
                # Only keep tournament urls labeled "CHESS.COM"
                if 'CHESS.COM' in element.text.upper():
                    url_list.append(url)
    return url_list     

In [80]:
import sqlite3
from sqlite3 import Connection, Cursor

# Drop then create tables for all data scraped by this module.
def init_db() -> None:
    conn: Connection = sqlite3.Connection('scrape_data.db')
    cur: Cursor = conn.cursor()
    cur.execute("""DROP TABLE IF EXISTS uscf_urls""")
    # Stores urls to allow us to navigate to all relevant USCF tournaments.
    cur.execute("""CREATE TABLE IF NOT EXISTS uscf_urls (
        id INTEGER PRIMARY KEY,
        date TEXT,
        url TEXT,
        scraped INTEGER
        )""")
    conn.commit()

In [81]:
# Scrapes a list of urls. Each url is a USCF tournament with a parallel entry
# on chess.com. Every month between 2015 and 2023 inclusive is checked.
def scrape_all_uscf_urls(driver: Chrome, cur: Cursor) -> List[str]:
    url_list: List[str] = []
    for year in range(2023, 2014, -1):
        # Page requires single digit months to have a 0 in front.
        for month in range(1, 10):
            date: str = '0' + str(month) + '/' + str(year)
            navigate_to_uscf_page(driver, date)
            url_list.extend(scrape_uscf_tournament_urls(driver))
        for month in range(10, 13):
            date: str = str(month) + '/' + str(year)
            navigate_to_uscf_page(driver, date)
            url_list.extend(scrape_uscf_tournament_urls(driver))
    return url_list

### TOURNAMENT OBSERVATIONS
INTEGER PRIMARY KEY
Round count
Player count
Number of rounds
Section date(s)
Received date
Entered date
Rated date
Re-Rated date
K Factor
Rating System
Tournament type
Time control
Tournament name
Tournament ID
Total points

### ROUNDS RESULT OBSERVATIONS
INTEGER PRIMARY KEY
TOURNAMENT FOREIGN KEY
Round number
Round result
Round opponent

### PLAYER OBSERVATIONS
INTEGER PRIMARY KEY
Real name
USCF before rating
USCF after rating
State
Pair number
Total points
USFC id


In [82]:
# from datetime import datetime

# STARTING_URL: str = 'https://www.chess.com/tournament/live?&page='

# def go_to_tournament(day, month, year, starting_url):
#     driver.get(starting_url)
#     date_string = driver.find_element(By.CLASS_NAME, 'tournaments-live-date')
#     date = datetime.strptime(date_string.text, '%b %d, %Y, %I:%M %p')

# 0 to 9000 is what chess.com currently allows
# def go_to_right_year(target, url, page) -> bool:
#     driver.get(url + page)
#     date_string = driver.find_element(By.CLASS_NAME, 'tournaments-live-date')
#     date = datetime.strptime(date_string.text, '%b %d, %Y, %I:%M %p')
#     if target < date.year:
#         page = page + step_size
#         step_size = step_size / 2

In [83]:
# driver: Chrome = load_driver()
# url_list: List[str] = scrape_all_uscf_urls(driver)

# url_tuples: List[Tuple] = []
# for url in url_list:
#     url_tuples.append((None, url, 0))

# conn: Connection = sqlite3.Connection('scrape_data.db')
# cur: Cursor = conn.cursor()
# cur.executemany("""INSERT INTO uscf_urls (
#     date, url, scraped
#     ) VALUES (?, ?, ? )""", url_tuples)
# conn.commit()

In [111]:
from bs4 import BeautifulSoup
from bs4.element import ResultSet, Tag
import requests
from requests.models import Response

cur: Cursor = sqlite3.Connection('scrape_data.db').cursor()
cur.execute("SELECT url FROM uscf_urls WHERE id = 1900")
url: str = cur.fetchall()[0][0]
request: Response = requests.get(url)
print(url)
print('Status Code:', request.status_code)
soup = BeautifulSoup(request.text, 'html.parser')

http://www.uschess.org/msa/XtblMain.php?201807095622
Status Code: 200


In [166]:
upper_table: ResultSet = soup.find_all('table', attrs={
    'border': '0',
    'width': '800',
    'bgcolor': 'FFFFFF',
    'cellpadding': '3',
    'cellspacing': '0'
})
rows: ResultSet = upper_table[0].find_all('tr')
row1_tags: ResultSet = rows[0].find_all('td')

name: str = row1_tags[3].b.text
tournament_id = row1_tags[3].small.text[1:-1]
event_date: str = row1_tags[7].b.text

dates_split: List[str] = row1_tags[13].b.text.split(' ')
received_date: str = dates_split[1]
entered_date: str = dates_split[4]
rated_date: str = dates_split[7]

section_count: int = int(row1_tags[15].b.text.split(' ')[0])
player_count: str = row1_tags[15].b.text.split(' ')[-2]

print(name)
print(tournament_id)
print(event_date)
print(received_date)
print(entered_date)
print(rated_date)
print(section_count)
print(player_count)

US CHESS BLITZ ON CHESS.COM #1
201807095622
2018-07-09 
2018-07-20
2018-07-20
2018-07-20
1
20


In [187]:
header_box: Tag = upper_table[1]
rules: str = header_box.find_all('b')[3].text
rules_list: List[str] = rules.split(' ')

k_factor: str = rules_list[7]
rating_system: str = rules_list[11]
tournament_type: str = rules_list[16]
time_control: str = rules_list[20]

print(k_factor)
print(rating_system)
print(tournament_type)
print(time_control)


F
OB
S
G/10;+0


In [190]:
s = soup.find_all('a', {'href': lambda x: x and x.startswith('MbrDtlMain')})
print(len(s))

20


In [160]:
pretty_soup = soup.prettify()
with open('page.txt', 'w') as fout:
    for line in pretty_soup:
        fout.write(line)